# Agent实战基础之MCP实践

## 1. MCP基础知识

### 1.1 uv基础介绍

uv入门基础知识
MCP开发要求借助uv进行虚拟环境创建和依赖管理。 uv 是一个Python 依赖管理工具，类似于
pip 和 conda ，但它更快、更高效，并且可以更好地管理 Python 虚拟环境和依赖项。它的核心目标是
替代 pip 、 venv 和 pip-tools ，提供更好的性能和更低的管理开销。

uv 的特点：
1. 速度更快：相比 pip ， uv 采用 Rust 编写，性能更优。
2. 支持 PEP 582：无需 virtualenv ，可以直接使用 __pypackages__ 进行管理。
3. 兼容 pip ：支持 requirements.txt 和 pyproject.toml 依赖管理。
4. 替代 venv ：提供 uv venv 进行虚拟环境管理，比 venv 更轻量。
5. 跨平台：支持 Windows、macOS 和 Linux。


**为什么MCP推荐使用uv进行环境管理？**

MCP 依赖的 Python 环境可能包含多个模块， uv 通过 pyproject.toml 提供更高效的管理方式，并且可以避免 pip 的一些依赖冲突问题。此外， uv 的包管理速度远超 pip ，这对于 MCP这样频繁管理依赖的项目来说是一个很大的优势。


## 1.2 uv常见命令

## 2. MCP client设计

### 2.1 创建简单的MCP Client项目

<center><img src="./pic/1.png" style="zoom:60%;" />

<center><img src="./pic/2.png" style="zoom:60%;" />

### 2.2 创建虚拟环境并激活

<center><img src="./pic/3.png" style="zoom:60%;" />

<center><img src="./pic/4.png" style="zoom:60%;" />

### 2.3 编写简易Client

<center><img src="./pic/5.png" style="zoom:60%;" />

In [2]:
import asyncio #支持异步操作
from mcp import ClientSession # MCP Client会话管理
from contextlib import AsyncExitStack  # 资源管理（确保客户端关闭时释放资源）

class MCPClient:
    def __init__(self):
        """初始化 MCP Client"""
        self.session = None
        self.exit_stack = AsyncExitStack()
        
    async def connect_to_mock_server(self):
        """模拟 MCP 服务器的连接（暂不连接真实服务器）"""
        print("MCP Client已初始化，但未连接到服务器")
        
    async def chat_loop(self):
        """运行交互式聊天循环"""
        print("\nMCP Client已启动！输入 'quit' 退出")
        while True:
            try:
                query = input("\nPrompt: ").strip()
                if query.lower() == 'quit':
                    break
                print(f"\n [模拟回复] 你说的是：{query}")
            except Exception as e:
                print(f"\n发生错误: {str(e)}")
                
    async def cleanup(self):
        """清理资源"""
        await self.exit_stack.aclose()
                
async def main():
    client = MCPClient()
    try:
        await client.connect_to_mock_server()
        await client.chat_loop()
    finally:
        await client.cleanup()
            
await main()
# if __name__ == "__main__":
    # asyncio.run(main())

MCP Client已初始化，但未连接到服务器

MCP Client已启动！输入 'quit' 退出

 [模拟回复] 你说的是：Enter

 [模拟回复] 你说的是：hello mcp

 [模拟回复] 你说的是：hi,what is mcp


### 2.4 运行客户端

uv run client_1.py

<center><img src="./pic/6.png" style="zoom:60%;" />

## 3. MCP Client接入大模型

### 3.1 添加依赖

<center><img src="./pic/7.png" style="zoom:60%;" />

### 3.2 创建隐藏文件.env

### 3.3 Client代码设计如下 

In [ ]:
import asyncio
import os
from openai import OpenAI
from dotenv import load_dotenv
from contextlib import AsyncExitStack
# 加载隐藏文件，确保 API Key 受到保护
load_dotenv()
class MCPClient:
    def __init__(self):
        """初始化 MCP Client"""
        self.exit_stack = AsyncExitStack()
        self.openai_api_key = os.getenv("OPENAI_API_KEY") # 读取 Key
        self.base_url = os.getenv("BASE_URL") # 读取 BASE URL
        self.model = os.getenv("MODEL") # 读取 model

        if not self.openai_api_key:
            raise ValueError("未找到 OpenAI API Key，请在 .env 文件中设置OPENAI_API_KEY")
        self.client = OpenAI(api_key=self.openai_api_key, base_url=self.base_url)
        
    async def process_query(self, query: str) -> str:
        """调用 OpenAI API 处理用户查询"""
        messages = [{"role": "system", "content": "你是一个智能助手，帮助用户回答问题。"},
                {"role": "user", "content": query}]
        try:
            # 调用大模型API
            response = await asyncio.get_event_loop().run_in_executor(
                
                None,lambda: self.client.chat.completions.create(
                model=self.model,
                messages=messages) )
            
            return response.choices[0].message.content
        except Exception as e:
            return f"调用 OpenAI API 时出错: {str(e)}"
            
    async def chat_loop(self):
        """运行交互式聊天循环"""
        print("\n MCP Client已启动！输入 'quit' 退出")
        
        while True:
            try:
                query = input("\n用户: ").strip()
                if query.lower() == 'quit':
                    break
                response = await self.process_query(query) 
                print(f"\n 智能助手: {response}")
            except Exception as e:
                print(f"\n 发生错误: {str(e)}")
                
    async def cleanup(self):
        """清理资源"""
        await self.exit_stack.aclose()
        
async def main():
    client = MCPClient()
    try:
        await client.chat_loop()
    finally:
        await client.cleanup()
if __name__ == "__main__":
    asyncio.run(main())

### 3.4 运行测试

In [ ]:
uv run client_2.py

<center><img src="./pic/8.png" style="zoom:60%;" />

## 4. MCP Server端设计

### 4.1 MCP Server概念介绍

根据MCP协议定义，Server可以提供三种类型的标准能力，Resources、Tools、Prompts，每个
Server可同时提供者三种类型能力或其中一种。

**Resources**：资源，类似于文件数据读取，可以是文件资源或是API响应返回的内容。

**Tools**：工具，第三方服务、功能函数，通过此可控制LLM可调用哪些函数。

**Prompts**：提示词，为用户预先定义好的完成特定任务的模板。

### 4.2 MCP Server的通讯机制

   Model Context Protocol（MCP）由Anthropic开源，用于将大型语言模型直接连接数据源。它支持**标准输入输出（stdio）** 和 **基于HTTP的服务器推送事件（SSE）** 两种传输方式。Stdio模式适用于本地通信，通过启动服务器作为子进程实现高效低延迟的数据交换，适合快速响应的本地应用。而基于HTTP和SSE的方式则适用于分布式或远程场景，实现客户端与服务器间实时数据推送。


    1. 本地通讯：使用了stdio传输数据，具体流程Client启动Server程序作为子进程，其消息通讯是通过stdin/stdout进行的，消息格式为JSON-RPC 2.0。
    
    2. 远程通讯：Client与Server可以部署在任何地方，Client使用SSE与Server进行通讯，消息的格式为JSON-RPC 2.0，Server定义了/see与/messages接口用于推送与接收数据

## 4.3 我们基于stdio实现一个【天气智能助手】

#### 4.3.1 天气智能助手流程设计 

<center><img src="./pic/9.png" style="zoom:60%;" />

#### 4.3.2 天气智能助手MCP服务端代码设计

In [ ]:
import json
import httpx
from typing import Any
from mcp.server.fastmcp import FastMCP

# 初始化 MCP 服务器
mcp = FastMCP("WeatherServer")
# OpenWeather API 配置
OPENWEATHER_API_BASE = "https://api.openweathermap.org/data/2.5/weather"
API_KEY = "5c939a7cc59eb8696fxxx75c5a9a" # 请替换为你自己的 OpenWeather API Key
USER_AGENT = "weather-app/1.0"

async def fetch_weather(city: str) -> dict[str, Any] | None:
    """
    从 OpenWeather API 获取天气信息。
    :param city: 城市名称（需使用英文，如 Beijing）
    :return: 天气数据字典；若出错返回包含 error 信息的字典
    """
    params = {
        "q": city,
        "appid": API_KEY,
        "units": "metric",
        "lang": "zh_cn"
     }
    headers = {"User-Agent": USER_AGENT}
    async with httpx.AsyncClient() as client:
        try:
            response = await client.get(OPENWEATHER_API_BASE, params=params,
            headers=headers, timeout=30.0)
            response.raise_for_status()
            return response.json() # 返回字典类型
        except httpx.HTTPStatusError as e:
            return {"error": f"HTTP 错误: {e.response.status_code}"}
        except Exception as e:
            return {"error": f"请求失败: {str(e)}"}
            
def format_weather(data: dict[str, Any] | str) -> str:
    """
    将天气数据格式化为易读文本。
    :param data: 天气数据（可以是字典或 JSON 字符串）
    :return: 格式化后的天气信息字符串
    """
    # 如果传入的是字符串，则先转换为字典
    if isinstance(data, str):
        try:
            data = json.loads(data)
        except Exception as e:
            return f"无法解析天气数据: {e}"
            # 如果数据中包含错误信息，直接返回错误提示
    if "error" in data:
        return f"{data['error']}"
    # 提取数据时做容错处理
    city = data.get("name", "未知")
    country = data.get("sys", {}).get("country", "未知")
    temp = data.get("main", {}).get("temp", "N/A")
    humidity = data.get("main", {}).get("humidity", "N/A")
    wind_speed = data.get("wind", {}).get("speed", "N/A")
    # weather 可能为空列表，因此用 [0] 前先提供默认字典
    weather_list = data.get("weather", [{}])
    description = weather_list[0].get("description", "未知")
    return (
        f"城市{city}, {country}\n"
        f"温度: {temp}°C\n"
        f"湿度: {humidity}%\n"
        f"风速: {wind_speed} m/s\n"
        f"天气: {description}\n")

@mcp.tool()
async def query_weather(city: str) -> str:
    """
    输入指定城市的英文名称，返回今日天气查询结果。
    :param city: 城市名称（需使用英文）
    :return: 格式化后的天气信息
    """
    data = await fetch_weather(city)
    return format_weather(data)

if __name__ == "__main__":
    mcp.run(transport='stdio')

#### 4.3.3 天气智能助手MCP客户端代码设计

In [ ]:
import asyncio
import os
import json
from typing import Optional
from contextlib import AsyncExitStack
from openai import OpenAI
from dotenv import load_dotenv
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

load_dotenv()

class MCPClient:
    def __init__(self):
        """初始化 MCP 客户端"""
        self.exit_stack = AsyncExitStack()
        self.openai_api_key = os.getenv("OPENAI_API_KEY") # 读取Key
        self.base_url = os.getenv("BASE_URL") # 读取 URL
        self.model = os.getenv("MODEL") # 读取 model
        if not self.openai_api_key:
            raise ValueError("未找到 OpenAI API Key")
        self.client = OpenAI(api_key=self.openai_api_key, base_url=self.base_url)
        
        self.session: Optional[ClientSession] = None
        self.exit_stack = AsyncExitStack()

    async def connect_to_server(self, server_script_path: str):
        """连接到 MCP 服务器并列出可用工具"""
        is_python = server_script_path.endswith('.py')
        is_js = server_script_path.endswith('.js')
        ## 判断服务器脚本是 Python 还是 Node.js，选择对应的运行命令
        if not (is_python or is_js):
            raise ValueError("服务器脚本必须是 .py 或 .js 文件")
        command = "python" if is_python else "node"
        ## 告诉 MCP 客户端如何启动服务器
        server_params = StdioServerParameters(
            command=command,
            args=[server_script_path],
            env=None
        )
        # 启动 MCP 服务器并建立通信
        stdio_transport = await self.exit_stack.enter_async_context(stdio_client(server_params))
        self.stdio, self.write = stdio_transport
        self.session = await self.exit_stack.enter_async_context(ClientSession(self.stdio, self.write))
        await self.session.initialize()
        # 向 MCP 服务器请求所有已注册的工具（用 @mcp.tool() 标记）
        response = await self.session.list_tools()
        tools = response.tools
        print("\n已连接到服务器，支持以下工具:", [tool.name for tool in tools])
 
    ## 这个本质就是Function call
    async def process_query(self, query: str) -> str:
        """
        使用大模型处理查询并调用可用的 MCP 工具 (Function Calling)
        """
        messages = [{"role": "user", "content": query}]
        response = await self.session.list_tools()
        available_tools = [{
            "type": "function",
            "function": {
            "name": tool.name,
            "description": tool.description,
            "input_schema": tool.inputSchema
            }
        } for tool in response.tools]
        # print(available_tools)
        response = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            tools=available_tools)
        # 处理返回的内容
        content = response.choices[0]
        if content.finish_reason == "tool_calls":
            # 如何是需要使用工具，就解析工具
            tool_call = content.message.tool_calls[0]
            tool_name = tool_call.function.name
            tool_args = json.loads(tool_call.function.arguments)
             # 执行工具
            result = await self.session.call_tool(tool_name, tool_args)
            print(f"\n\n[Calling tool {tool_name} with args {tool_args}]\n\n")
            # 将模型返回的调用哪个工具数据和工具执行完成后的数据都存入messages中
            messages.append(content.message.model_dump())
            messages.append({
                "role": "tool",
                "content": result.content[0].text,
                "tool_call_id": tool_call.id,})
            
            # 将上面的结果再返回给大模型用于生产最终的结果
            response = self.client.chat.completions.create(
                model=self.model,
                messages=messages,
            )
            return response.choices[0].message.content
        return content.message.content
        
    async def chat_loop(self):
        """运行交互式聊天循环"""
        print("\n 智能助手客户端已启动！输入 'quit' 退出")
        while True:
            try:
                query = input("\n用户: ").strip()
                if query.lower() == 'quit':
                    break
                response = await self.process_query(query) # 发送用户输入到 OpenAIAPI
                print(f"\n天气预报智能助手: {response}")
            except Exception as e:
                print(f"\n发生错误: {str(e)}")
                
    async def cleanup(self):
        """清理资源"""
        await self.exit_stack.aclose()
            
async def main():
    if len(sys.argv) < 2:
        print("Usage: python client.py <path_to_server_script>")
        sys.exit(1)
    client = MCPClient()
    try:
        await client.connect_to_server(sys.argv[1])
        await client.chat_loop()
    finally:
        await client.cleanup()
    
if __name__ == "__main__":
    import sys
    asyncio.run(main())

#### 4.3.4 流程测试

In [ ]:
uv run client_3.py server.py

<center><img src="./pic/10.png" style="zoom:60%;" />

## 5. MCP Inspector功能介绍

   在实际开发MCP服务器的过程中，Anthropic提供了一个非常便捷的debug工具：Inspector。借助
Inspector，我们能够非常快捷的调用各类server，并测试其功能

### 5.1 安装node.js 

In [ ]:
curl -fsSL https://deb.nodesource.com/setup_20.x | sudo bash -
sudo apt install -y nodejs

<center><img src="./pic/11.png" style="zoom:60%;" />

<center><img src="./pic/12.png" style="zoom:60%;" />

### 5.2 启动Inspector

In [ ]:
npx -y @modelcontextprotocol/inspector uv run server.py

<center><img src="./pic/13.png" style="zoom:60%;" />

In [ ]:
autodl是云平台，我们直接访问平台，需要做端口映射

<center><img src="./pic/14.png" style="zoom:60%;" />

In [ ]:
工具测试

<center><img src="./pic/15.png" style="zoom:60%;" />

<center><img src="./pic/15.png" style="zoom:60%;" />

<center><img src="./pic/16.png" style="zoom:60%;" />

<center><img src="./pic/17.png" style="zoom:60%;" />

<center><img src="./pic/18.png" style="zoom:60%;" />

## 扩展

Anthropic MCP发布：https://www.anthropic.com/news/model-context-protocol

官方Server: https://github.com/modelcontextprotocol/servers